In [87]:
import pandas as pd
import numpy as np

# 1. 다중 인덱스된 Series
어떻게 하면 2차원 데이터를 1차원 Series에 표현할 수 있을까?

### 1-1 나쁜 방식

In [88]:
index = [('California', 2000), ('California', 2010),
        ('New York', 2000), ('New York', 2010),
        ('Texas', 2000), ('Texas', 2010)]

populations = [3387148, 37253956,
              18976457, 19378102,
              20851820, 25145561]
pop = pd.Series(populations, index = index)
pop

(California, 2000)     3387148
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [89]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

In [90]:
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

In [91]:
[i for i in pop.index if i[1] == 2010]

[('California', 2010), ('New York', 2010), ('Texas', 2010)]

### 1-2 더 나은 방식: Pandas MultiIndex

In [92]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
           labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

In [93]:
pop = pop.reindex(index)
pop

California  2000     3387148
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [94]:
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

### 1-3 MultiIndex: 추가 지원

In [95]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,3387148,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [96]:
pop_df.stack()

California  2000     3387148
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [97]:
pop_df = pd.DataFrame({'total': pop,
                      'under18': [9267089, 9284094,
                                 4687374, 4318033,
                                 5906301, 6879014]})
pop_df

total  under18
California 2000   3387148  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In [98]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18

California  2000    2.735956
            2010    0.249211
New York    2000    0.247010
            2010    0.222831
Texas       2000    0.283251
            2010    0.273568
dtype: float64

In [99]:
f_u18.unstack()

,2000,2010
California,2.735956,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


# 2. MultiIndex 생성 메서드

In [100]:
# 다중 인덱스를 가진 Series나 DataFrame을 생성하는 가장 간단한 방식은 생성자에게 2개 이상의 인덱스 배열 리스트를 전달하는 것이다.

df = pd.DataFrame(np.random.rand(4,2),
                 index = [['a', 'a', 'b', 'b'], [1,2,1,2]],
                 columns = ['data1', 'data2'])
df

data1     data2
a 1  0.983768  0.444716
  2  0.022590  0.860346
b 1  0.038628  0.421887
  2  0.097131  0.748767

In [101]:
# 위와 비슷하게 튜플을 키로 갖는 딕셔너리를 전달하면 Pandas는 자동으로 이것을 인식해 기본으로 MultiIndex를 사용한다.
data = {('California', 2000):3387148,
        ('California', 2010):37253956,
        ('New York', 2000):18976457,
        ('New York', 2010):19378102,
        ('Texas', 2000):20851820,
        ('Texas', 2010):25145561}
pd.Series(data)

California  2000     3387148
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### 2-1 명시적 MultiIndex 생성자

In [102]:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [103]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [104]:
pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
            labels = [[0, 0, 1, 1], [0, 1, 0, 1]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

### 2-2 MultiIndex 레벨 이름

In [105]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000     3387148
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### 2-3 열의 MultiIndex
행이 인덱스의 여러 레벨을 가질 수 있듯이 열도 여러 레벨을 가질 수 있다.

In [106]:
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names = ['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                    names = ['subject', 'type'])

data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

health_data = pd.DataFrame(data, index = index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      42.0  37.1  48.0  37.9  39.0  38.1
     2      33.0  37.8  35.0  39.5  44.0  36.1
2014 1      45.0  35.6  50.0  36.9  25.0  36.8
     2      45.0  38.4  29.0  37.1  46.0  36.4

In [107]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      48.0  37.9
     2      35.0  39.5
2014 1      50.0  36.9
     2      29.0  37.1

# 3. MultiIndex 인덱싱 및 슬라이싱

### 3-1 다중 인덱스를 가진 Series

In [108]:
pop

state       year
California  2000     3387148
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [109]:
pop['California', 2000]

3387148

In [110]:
pop['California']

year
2000     3387148
2010    37253956
dtype: int64

In [111]:
pop.loc['California':'New York']

state       year
California  2000     3387148
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

In [112]:
pop[:, 2000]

state
California     3387148
New York      18976457
Texas         20851820
dtype: int64

In [113]:
# 부울 마스크를 이용해 데이터를 선택할 수도 있음.
pop[ pop > 22000000]

state       year
California  2010    37253956
Texas       2010    25145561
dtype: int64

In [114]:
# 팬시 인덱싱을 이용한 데이터 선택도 가능함.
pop[['California', 'Texas']]

state       year
California  2000     3387148
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

### 3-2 다중 인덱스를 가진 DataFrame

In [115]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      42.0  37.1  48.0  37.9  39.0  38.1
     2      33.0  37.8  35.0  39.5  44.0  36.1
2014 1      45.0  35.6  50.0  36.9  25.0  36.8
     2      45.0  38.4  29.0  37.1  46.0  36.4

In [116]:
health_data['Guido', 'HR']

year  visit
2013  1        48.0
      2        35.0
2014  1        50.0
      2        29.0
Name: (Guido, HR), dtype: float64

In [117]:
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      42.0  37.1
     2      33.0  37.8

In [118]:
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        42.0
      2        33.0
2014  1        45.0
      2        45.0
Name: (Bob, HR), dtype: float64

In [119]:
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,42.0,48.0,39.0
2014,1,45.0,50.0,25.0


# 4. 다중 인덱스 재정렬하기

### 4-1 정렬된 인덱스와 정렬되지 않은 인덱스

In [120]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index = index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.707604
      2      0.358685
c     1      0.976405
      2      0.673225
b     1      0.882089
      2      0.511024
dtype: float64

In [121]:
# 대부분의 MultiIndex 슬라이싱 연산은 인덱스가 정렬돼 있지 않으면 실패한다.

try:
    data['a': 'b']
except KeyError as e:
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


In [122]:
data = data.sort_index()
data

char  int
a     1      0.707604
      2      0.358685
b     1      0.882089
      2      0.511024
c     1      0.976405
      2      0.673225
dtype: float64

In [123]:
data['a': 'b']

char  int
a     1      0.707604
      2      0.358685
b     1      0.882089
      2      0.511024
dtype: float64

### 4-2 인덱스 스태킹 및 언스태킹

In [124]:
pop

state       year
California  2000     3387148
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [125]:
pop.unstack(level = 0)

state,California,New York,Texas
year,,,
2000,3387148,18976457,20851820
2010,37253956,19378102,25145561


In [126]:
pop.unstack(level = 1)

year,2000,2010
state,,
California,3387148,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [127]:
pop.unstack().stack()

state       year
California  2000     3387148
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### 4-3 인덱스 및 재설정

In [128]:
pop_flat = pop.reset_index(name = 'population')
pop_flat

,state,year,population
0,California,2000,3387148
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [129]:
pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000     3387148
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

# 5. 다중 인덱스에서 데이터 집계

In [130]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      42.0  37.1  48.0  37.9  39.0  38.1
     2      33.0  37.8  35.0  39.5  44.0  36.1
2014 1      45.0  35.6  50.0  36.9  25.0  36.8
     2      45.0  38.4  29.0  37.1  46.0  36.4

In [131]:
data_mean = health_data.mean(level='year')
data_mean

subject   Bob        Guido         Sue      
type       HR   Temp    HR  Temp    HR  Temp
year                                        
2013     37.5  37.45  41.5  38.7  41.5  37.1
2014     45.0  37.00  39.5  37.0  35.5  36.6

In [132]:
data_mean.mean(axis=1, level='type')

type,HR,Temp
year,,
2013,40.166667,37.750000
2014,40.000000,36.866667
